# US01 - 4 
## Python
### Game of thrones part 2

Know, we have our first graph. But there is some problem. For example we have 'Waymar Royce', 'Royce', 'Ser Waymar' and 'Waymar' , there are all the same person. We need to find a way to regroup them.

1. Similarity in graph 
2. Word2vec (gensim) similiarities
3. Ask for the list of characters and compare with our
    We can find one here: https://awoiaf.westeros.org/index.php/List_of_characters 
  
**1. Lemmatize**

We will to use the third way. We need to lemmatize (lemma: there most complete version of the word, here the name). We call an API and clean the data (we use BeautifulSoup to get the information we want) to get the list of characters  

In [6]:
import requests

url = 'https://awoiaf.westeros.org/index.php/List_of_characters'
r = requests.get(url)

text = r.text

In [43]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(text, 'lxml')

links = soup.find_all('a') # <a href='url'>text<\a>

#our array for the page which mention a characters
pages = []

for link in links:
    what_we_want = set(['title', 'href'])
    what_there_is = set(link.attrs.keys())
    # take only the link for a characters (which have exactly title and href for attribues)
    if what_we_want == what_there_is:
        # we make tuples in an array
        pages.append((link['href'],link['href'].split('/')[-1]))
        
# we have to remove the last 5
pages = pages[:-5]
#remove the duplicate
pages = set(pages)
print('length of the pages is', len(pages))

length of the pages is 2118


There is pages 2126 to download. It's will take around 3h. Download during 3h is tricky in China. So we need to download one by one and if something go wrong, start again where the script stop. Here, we just try with one page: Aegon V Targaryen, because it'll be to long otherwise. 

In [51]:
for href, name in pages: 
    if name =='Aegon_V_Targaryen':
        print(href)
        r = requests.get('https://awoiaf.westeros.org' + href)
        with open(name+'.html', 'w+', encoding = 'utf-8') as fp:
            fp.write(r.text)

/index.php/Aegon_V_Targaryen


Now we have the page about Aegon V Targaryen. We need to find the name, the surname, the full name... This information are in the *infobox* (the box in the right like wikipedia). Here, the information of the infobox are in the *tbody*. 

In [55]:
with open('Aegon_V_Targaryen.html', 'r', encoding = 'utf-8') as fp: 
    html = fp.read()
soup = BeautifulSoup(html, 'lxml')

soup.find("table", class_="infobox").b.text


'Aegon V'

In [60]:
# save all the tbody (info in the infobox)
tbody = soup.find("table", class_="infobox").tbody

# take all the row in the tbody 
rows = tbody.find_all("tr")

# parcours the tree
# parcours all the tr, tr contains td or th and td
for row in rows:
    # so the child of row (tr) are td or th
    for child in row.children: 
        if child.name =='th': #here, th = full name, td = the name
            if row.th.text == 'Full Name': # get the td (the name)
                name = row.td.text
            if row.th.text == 'Alias': # get the td (the name)
                alias = row.td.text    

Problem is that our alias are stick with each other (Lower casse follow by Upper casse). We use a regular expression to modify (try on https://regex101.com/)

In [62]:
import re

#if a lower casse letter is immediately follow by a upper casse letter
regex = r"([a-z])([A-Z])"
subst = "\\1, \\2"

result = re.sub(regex, subst, alias)
print(result)

#put the data in a array, ready to be used
print([t.strip() for t in result.split(',')])


Aegon the Unlikely, Aegon the Fortunate, Egg, The Prince Who Was An Egg
['Aegon the Unlikely', 'Aegon the Fortunate', 'Egg', 'The Prince Who Was An Egg']


Know we are trying to get the list of books where the characters appears and clean the data 

In [72]:
for row in rows:
    for child in row.children: 
        if child.name =='th': 
            if row.th.text == 'Books': 
                books = row.td.text
print(books)

#delete the "()" and split the data 
import re

regex = r"\([a-zA-Z]+\)"
subst = ","

result = re.sub(regex, subst, books)
print(result)

#put the data in a array, ready to be used
print([t.strip() for t in result.split(',')])




The World of Ice & Fire (mentioned)The Hedge Knight (appears)The Sworn Sword (appears)The Mystery Knight (appears)A Game of Thrones (mentioned)A Clash of Kings (mentioned)A Storm of Swords (mentioned)A Feast for Crows (mentioned)A Dance with Dragons (mentioned)

The World of Ice & Fire ,The Hedge Knight ,The Sworn Sword ,The Mystery Knight ,A Game of Thrones ,A Clash of Kings ,A Storm of Swords ,A Feast for Crows ,A Dance with Dragons ,
['The World of Ice & Fire', 'The Hedge Knight', 'The Sworn Sword', 'The Mystery Knight', 'A Game of Thrones', 'A Clash of Kings', 'A Storm of Swords', 'A Feast for Crows', 'A Dance with Dragons', '']


### *The problem is there is a space before the "(" so we can split one the lowercasse follow by the uppercasse. So I put a ',' to split and consequently I have a empty book at the end...*